In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
import graphviz

In [2]:
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from sklearn.metrics import classification_report

from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_text
from sklearn.tree import export_graphviz

In [3]:
np.set_printoptions(precision=2)

In [4]:
df = pd.read_csv('all_samples.csv')
df.drop(columns=['Unnamed: 0', 'name'], inplace=True)
df.head()

,ILMN_1651217,ILMN_1651229,ILMN_1651234,ILMN_1651236,ILMN_1651237,ILMN_1651254,ILMN_1651259,ILMN_1651260,ILMN_1651261,ILMN_1651262,...,ILMN_1815885,ILMN_1815908,ILMN_1815923,ILMN_1815924,ILMN_1815933,ILMN_1815937,ILMN_1815938,ILMN_1815941,ILMN_1815951,CELIAC
0,4.229567,4.802085,4.145582,4.274502,4.268115,6.853804,4.401350,4.123169,4.639975,7.136778,...,4.376735,4.395501,4.338936,5.198647,4.594269,4.264604,4.256310,4.821757,5.005588,1
1,4.197183,4.820311,4.171221,4.332524,4.186809,6.663657,4.559615,4.278860,4.994493,6.803521,...,4.732124,4.417266,4.656831,4.615440,4.594269,4.336589,4.317376,4.518347,4.308311,1
2,4.131493,4.640774,4.075849,4.233316,4.334549,6.694727,4.370504,4.169419,5.093272,6.720391,...,4.292552,4.379864,4.211071,5.530672,4.570808,4.379545,4.241886,4.680351,4.780989,1
3,4.207410,4.508425,4.100585,4.166837,4.530517,6.506971,4.483179,4.242860,5.138309,6.881151,...,4.371180,4.406084,4.186757,5.358646,4.632107,4.282658,4.237614,4.602680,4.637598,1
4,4.245230,4.538779,4.040637,4.266853,4.326313,6.774611,4.409940,4.228860,4.948306,6.847382,...,4.345227,4.488653,4.364008,5.605900,4.624200,4.275774,4.251683,4.686359,4.687048,1


In [5]:
input_cols = list(df.columns[:-1])
inputs = df[input_cols]
target = df['CELIAC']

In [6]:
X_train, X_test, Y_train, Y_test = train_test_split(inputs, target, stratify=target)
# default test/train split 75/25

In [7]:
tree_gini = DecisionTreeClassifier(random_state=0)
tree_entropy = DecisionTreeClassifier(criterion='entropy', random_state=0)

In [8]:
models = [('Gini', tree_gini), 
          ('Entropy', tree_entropy)
         ]

In [9]:
# precision = TP / (TP + FP)
# recall = TP / (TP + FN)
# f1-score: harmonic mean of precision and recall

for name, model in models:
    print(name)
    model = model.fit(X_train, Y_train)
    prediction = model.predict(X_test)
    print(classification_report(Y_test, prediction,
                               zero_division=0))
    print(confusion_matrix(Y_test, prediction, 
                           labels=model.classes_,
                          normalize='all'))
    print('\n')

Gini
              precision    recall  f1-score   support

           0       0.14      0.17      0.15         6
           1       0.81      0.78      0.79        27

    accuracy                           0.67        33
   macro avg       0.48      0.47      0.47        33
weighted avg       0.69      0.67      0.68        33

[[0.03 0.15]
 [0.18 0.64]]


Entropy
              precision    recall  f1-score   support

           0       0.60      0.50      0.55         6
           1       0.89      0.93      0.91        27

    accuracy                           0.85        33
   macro avg       0.75      0.71      0.73        33
weighted avg       0.84      0.85      0.84        33

[[0.09 0.09]
 [0.06 0.76]]




In [10]:
# Use entropy (information gain)
# All examples (no test/train split)
tree = DecisionTreeClassifier(criterion='entropy', random_state=0)
tree = tree.fit(inputs, target)

In [11]:
txt = export_text(tree, feature_names=list(df.columns[:-1]))
print(txt)

|--- ILMN_1716080 <= 5.52
|   |--- ILMN_1705686 <= 5.70
|   |   |--- class: 0
|   |--- ILMN_1705686 >  5.70
|   |   |--- class: 1
|--- ILMN_1716080 >  5.52
|   |--- ILMN_1752895 <= 5.18
|   |   |--- class: 0
|   |--- ILMN_1752895 >  5.18
|   |   |--- ILMN_1754325 <= 5.52
|   |   |   |--- ILMN_1725346 <= 4.23
|   |   |   |   |--- class: 1
|   |   |   |--- ILMN_1725346 >  4.23
|   |   |   |   |--- class: 0
|   |   |--- ILMN_1754325 >  5.52
|   |   |   |--- class: 0



In [13]:
g = export_graphviz(tree, out_file=None,
                   class_names=True,
                   feature_names=list(df.columns[:-1]),
                   filled=True)

In [14]:
graph = graphviz.Source(g)

In [15]:
# graph.render("celiac_tree")
graph.render("celiac_tree_allsamples")

'celiac_tree_allsamples.pdf'

In [16]:
lookup = pd.read_csv('gene_lookup.csv')
lookup.drop(columns=['Unnamed: 0'], inplace=True)
lookup.head()

,ID,RefSeq_ID,Symbol,Definition
0,ILMN_1698220,NM_020432.2,PHTF2,Homo sapiens putative homeodomain transcriptio...
1,ILMN_1810835,NM_005416.1,SPRR3,Homo sapiens small proline-rich protein 3 (SPR...
2,ILMN_1782944,NM_004767.2,GPR37L1,Homo sapiens G protein-coupled receptor 37 lik...
3,ILMN_1692858,NM_012173.3,FBXO25,"Homo sapiens F-box protein 25 (FBXO25), transc..."
4,ILMN_1668162,NM_001013579.1,DGAT2L3,Homo sapiens diacylglycerol O-acyltransferase ...


In [19]:
q = lookup[lookup['ID'] == 'ILMN_1716080']

In [20]:
q['Symbol'].to_string(index=False)

' CBL'

In [21]:
def probe_to_gene(probe: str) -> str:
    q = lookup[lookup['ID'] == probe]
    if len(q) > 0:
        return q['Symbol'].to_string(index=False)
    return "?"

In [22]:
gene_symbols = [probe_to_gene(p) for p in list(df.columns[:-1])]

In [24]:
gene_symbols[:3]

[' PDCD1LG2', ' IPO13', ' SYT14']

In [26]:
g = export_graphviz(tree, out_file=None,
                    class_names=True,
                    feature_names=gene_symbols,
                    filled=True)

In [27]:
graph = graphviz.Source(g)

In [28]:
graph.render("celiac_tree_allsamples_genes")

'celiac_tree_allsamples_genes.pdf'